<a href="https://colab.research.google.com/github/alexander-toschev/ml-cs-intro/blob/main/practise/TF_Practice_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/alexander-toschev/ml-cs-intro/blob/main/home-work/HW_END_TO_END.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Practice Notebook 🇬🇧

This Colab notebook is for **hands-on practice** with TensorFlow 2.x and `tf.keras`.

It contains several *guided exercises*:
1. Basic TensorFlow tensors & operations  
2. Simple dense neural network on MNIST  
3. Convolutional neural network (CNN) on MNIST  
4. `tf.data` input pipelines  
5. Custom training loop with `tf.GradientTape`  

> Fill in all places marked with `# TODO` and run the cells.  
> There is **no auto-grading** here — this is pure practice.


## 0. Setup and imports

In this section we import TensorFlow and load the MNIST dataset.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

print("TensorFlow version:", tf.__version__)

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Add channel dimension: (N, 28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

num_classes = 10
input_shape = x_train.shape[1:]

print("Train shape:", x_train.shape, "Test shape:", x_test.shape)


TensorFlow version: 2.19.0
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Train shape: (60000, 28, 28, 1) Test shape: (10000, 28, 28, 1)


## 1. Practice: Basic tensors and operations

In this exercise, you will:

- Create random tensors
- Compute basic statistics with TensorFlow ops
- Reshape and slice tensors

### Task 1.1

Create a function `create_random_tensor(n)` that:

- Takes an integer `n`
- Returns a 1D tensor of shape `(n,)` with values sampled from a normal distribution `N(0, 1)` **using TensorFlow**

### Task 1.2

Create a function `describe_tensor(x)` that:

- Takes a 1D tensor `x`
- Prints:
  - shape
  - dtype
  - mean
  - standard deviation
  - min and max values


In [2]:
# TODO: implement create_random_tensor and describe_tensor

def create_random_tensor(n: int) -> tf.Tensor:
    """Return a 1D tensor of shape (n,) sampled from N(0, 1)."""
    # TODO: use TensorFlow to create random normal values
    # hint: tf.random.normal(...)
    x = tf.random.normal(shape=(n,), mean=0.0, stddev=1.0, seed=SEED)
    return x


def describe_tensor(x: tf.Tensor) -> None:
    """Print basic information about the tensor x."""
    # TODO: print shape, dtype, mean, std, min, max using TF ops
    print("Shape:", x.shape)
    print("Dtype:", x.dtype)
    mean = tf.reduce_mean(x)
    std = tf.math.reduce_std(x)
    min_v = tf.reduce_min(x)
    max_v = tf.reduce_max(x)
    print("Mean:", float(mean.numpy()))
    print("Std: ", float(std.numpy()))
    print("Min: ", float(min_v.numpy()))
    print("Max: ", float(max_v.numpy()))


# Try it
x = create_random_tensor(10)
describe_tensor(x)


Shape: (10,)
Dtype: <dtype: 'float32'>
Mean: 0.28105780482292175
Std:  0.6738396286964417
Min:  -0.7991440296173096
Max:  1.3148772716522217


## 2. Practice: Dense neural network on MNIST

Now you will build a simple **fully-connected (dense) network** for MNIST classification.

### Task 2.1 — Build the model

Implement `build_dense_model(input_shape, num_classes)`:

- Input: `input_shape = (28, 28, 1)`
- Architecture (recommended):
  - `Flatten`
  - `Dense(256, activation="relu")`
  - `Dense(128, activation="relu")`
  - `Dense(num_classes, activation="softmax")`
- Compile with:
  - `Adam(1e-3)`
  - `"sparse_categorical_crossentropy"`
  - metric `"accuracy"`

### Task 2.2 — Train the model

Train the model for **5 epochs** with batch size 128 and use `validation_split=0.1`.

Observe:
- Training / validation accuracy
- Underfitting or overfitting?


In [3]:
# TODO: implement build_dense_model and train it

def build_dense_model(input_shape, num_classes):
    """Build and compile a dense neural network for MNIST."""
    # TODO: define the model using tf.keras
    inputs = keras.Input(shape=input_shape)
    x = keras.layers.Flatten()(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


dense_model = build_dense_model(input_shape, num_classes)
dense_model.summary()

history_dense = dense_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=128,
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.8503 - loss: 0.5402 - val_accuracy: 0.9677 - val_loss: 0.1102
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9630 - loss: 0.1257 - val_accuracy: 0.9750 - val_loss: 0.0862
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9763 - loss: 0.0776 - val_accuracy: 0.9760 - val_loss: 0.0800
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9848 - loss: 0.0523 - val_accuracy: 0.9753 - val_loss: 0.0799
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9899 - loss: 0.0352 - val_accuracy: 0.9762 - val_loss: 0.0834


## 3. Practice: Convolutional Neural Network (CNN) on MNIST

Now build a small **CNN** which usually works better for images.

### Task 3.1 — Build a CNN model

Implement `build_cnn_model(input_shape, num_classes)` with:

- `Conv2D(32, kernel_size=3, activation="relu")`
- `MaxPooling2D()`
- `Conv2D(64, kernel_size=3, activation="relu")`
- `MaxPooling2D()`
- `Flatten`
- `Dense(128, activation="relu")`
- `Dense(num_classes, activation="softmax")`

Compile the model with the same settings as before.

### Task 3.2 — Train and compare

Train for **5 epochs** and compare validation accuracy with the dense model.


In [4]:
# TODO: implement build_cnn_model and train it

def build_cnn_model(input_shape, num_classes):
    """Build and compile a simple CNN for MNIST."""
    inputs = keras.Input(shape=input_shape)
    x = keras.layers.Conv2D(32, 3, activation="relu")(inputs)
    x = keras.layers.MaxPooling2D()(x)
    x = keras.layers.Conv2D(64, 3, activation="relu")(x)
    x = keras.layers.MaxPooling2D()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


cnn_model = build_cnn_model(input_shape, num_classes)
cnn_model.summary()

history_cnn = cnn_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=128,
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8529 - loss: 0.5130 - val_accuracy: 0.9818 - val_loss: 0.0682
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9810 - loss: 0.0634 - val_accuracy: 0.9878 - val_loss: 0.0469
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9872 - loss: 0.0416 - val_accuracy: 0.9897 - val_loss: 0.0393
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9908 - loss: 0.0294 - val_accuracy: 0.9897 - val_loss: 0.0367
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9933 - loss: 0.0225 - val_accuracy: 0.9895 - val_loss: 0.0408


## 4. Practice: `tf.data` input pipeline

In this section, you will build a `tf.data.Dataset` for MNIST and use it with `.fit()`.

### Task 4.1 — Create datasets

Implement `make_dataset(x, y, batch_size)` that:

- creates a `tf.data.Dataset` from `(x, y)`
- shuffles with buffer size `10000`
- batches with `batch_size`
- prefetches with `tf.data.AUTOTUNE`

Create:

- `train_ds` from `x_train, y_train`
- `test_ds` from `x_test, y_test` (without shuffling)

### Task 4.2 — Train with `tf.data`

Train the **CNN model** from the previous section using `train_ds` instead of NumPy arrays.


In [ ]:
# TODO: implement make_dataset and train with tf.data

def make_dataset(x, y, batch_size: int, shuffle: bool = True) -> tf.data.Dataset:
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=10000, seed=SEED)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


batch_size = 128
train_ds = make_dataset(x_train, y_train, batch_size=batch_size, shuffle=True)
test_ds = make_dataset(x_test, y_test, batch_size=batch_size, shuffle=False)

# Rebuild a fresh CNN model
cnn_model_ds = build_cnn_model(input_shape, num_classes)

history_cnn_ds = cnn_model_ds.fit(
    train_ds,
    epochs=5,
    validation_data=test_ds,
)


## 5. Practice: Custom training loop with `tf.GradientTape`

Here we will train a small network for a **toy regression problem** using a fully custom loop.

We will:

1. Generate synthetic data for `y = 3x + noise`
2. Build a small dense model with Keras
3. Write the training loop using `tf.GradientTape`

### Task 5.1 — Generate data

- Create 1000 points `x` uniformly in [-1, 1]
- Compute `y = 3 * x + noise`, where noise is normal with std 0.1

### Task 5.2 — Build model

- Simple `Sequential` model:
  - `Dense(8, activation="relu", input_shape=(1,))`
  - `Dense(1)`

### Task 5.3 — Custom loop

For several epochs:

- for each batch:
  - run forward pass
  - compute MSE loss
  - compute gradients w.r.t. trainable variables
  - apply gradients with Adam optimizer

Observe how loss decreases and how learned weight compares to 3.0.


In [ ]:
# TODO: implement the custom training loop for regression

# 5.1 Generate data
n_samples = 1000
x_reg = np.random.uniform(-1.0, 1.0, size=(n_samples, 1)).astype("float32")
noise = np.random.normal(loc=0.0, scale=0.1, size=(n_samples, 1)).astype("float32")
y_reg = 3.0 * x_reg + noise

# Build dataset
batch_size_reg = 32
reg_ds = tf.data.Dataset.from_tensor_slices((x_reg, y_reg))
reg_ds = reg_ds.shuffle(buffer_size=1000, seed=SEED).batch(batch_size_reg)

# 5.2 Build model
reg_model = keras.Sequential([
    keras.layers.Dense(8, activation="relu", input_shape=(1,)),
    keras.layers.Dense(1),
])

optimizer = keras.optimizers.Adam(learning_rate=1e-2)
loss_fn = keras.losses.MeanSquaredError()

# 5.3 Custom training loop
n_epochs_reg = 20

for epoch in range(n_epochs_reg):
    epoch_loss = 0.0
    n_batches = 0

    for x_batch, y_batch in reg_ds:
        with tf.GradientTape() as tape:
            y_pred = reg_model(x_batch, training=True)
            loss_value = loss_fn(y_batch, y_pred)

        grads = tape.gradient(loss_value, reg_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, reg_model.trainable_variables))

        epoch_loss += float(loss_value.numpy())
        n_batches += 1

    epoch_loss /= n_batches
    print(f"Epoch {epoch+1:02d}: loss = {epoch_loss:.4f}")

# Inspect learned weight (approx. 3.0)
for var in reg_model.trainable_variables:
    print(var.name, var.numpy())
